<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/NLP/4-5-Document-loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Loading

## Retrieval augmented generation

검색 증강 생성(RAG)에서 LLM은 실행의 일부로 외부 데이터세트에서 상황별 문서를 검색합니다.

특정 문서(예: PDF, 비디오 세트 등)에 대해 질문하려는 경우에 유용합니다.

In [1]:
!pip install openai==0.28
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.2 MB/s eta 0:00:00


In [8]:
! pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

## PDFs

Andrew Ng의 유명한 CS229 과정의 PDF이고 이러한 문서는 자동 전사의 결과이므로 단어와 문장이 예기치 않게 분리되는 경우가 있습니다.

In [5]:
! pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 18.6 MB/s eta 0:00:00


In [6]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1pqkdvOpsiPJT7pf-rhkYHN6fgIiXyUHR' -O MachineLearning-Lecture01.pdf


--2024-10-08 06:31:17--  https://drive.google.com/uc?export=download&id=1pqkdvOpsiPJT7pf-rhkYHN6fgIiXyUHR
Resolving drive.google.com (drive.google.com)... 74.125.130.102, 74.125.130.101, 74.125.130.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.130.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1pqkdvOpsiPJT7pf-rhkYHN6fgIiXyUHR&export=download [following]
--2024-10-08 06:31:17--  https://drive.usercontent.google.com/download?id=1pqkdvOpsiPJT7pf-rhkYHN6fgIiXyUHR&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.12.132, 2404:6800:4003:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.12.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64553 (63K) [application/octet-stream]
Saving to: ‘MachineLearning-Lecture01.pdf’

MachineLearning-Lec 100%[===================>]  63.0

In [9]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("MachineLearning-Lecture01.pdf")
pages = loader.load()

Each page is a `Document`.

A `Document` contains text (`page_content`) and `metadata`.

In [10]:
len(pages)

22

In [11]:
page = pages[0]

In [12]:
page

Document(metadata={'source': 'MachineLearning-Lecture01.pdf', 'page': 0}, page_content='MachineLearning-Lecture01  \nInstructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is ju st spend a little time going over the logistics \nof the class, and then we\'ll start to  talk a bit about machine learning.  \nBy way of introduction, my name\'s  Andrew Ng and I\'ll be instru ctor for this class. And so \nI personally work in machine learning, and I\' ve worked on it for about 15 years now, and \nI actually think that machine learning is th e most exciting field of all the computer \nsciences. So I\'m actually always excited about  teaching this class. Sometimes I actually \nthink that machine learning is not only the most exciting thin g in computer science, but \nthe most exciting thing in all of human e ndeavor, so maybe a little bias there.  \nI also want to introduce the TAs, who are all graduate students doing research in or

In [13]:
print(page.page_content[0:500])

MachineLearning-Lecture01  
Instructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is ju st spend a little time going over the logistics 
of the class, and then we'll start to  talk a bit about machine learning.  
By way of introduction, my name's  Andrew Ng and I'll be instru ctor for this class. And so 
I personally work in machine learning, and I' ve worked on it for about 15 years now, and 
I actually think that machine learning i


In [14]:
page.metadata

{'source': 'MachineLearning-Lecture01.pdf', 'page': 0}

OpenAIEmbeddings

In [15]:
!pip install tiktoken
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.4 MB/s eta 0:00:00


In [16]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("How will the community be engaged?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

<ipython-input-16-8cfad4d3c268>:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())


5: So all right, online resources. The class has a home page, so it's in on the handouts. I 
won't write on the chalkboard — http:// cs229.stanford.edu. And so when there are 
homework assignments or things like that, we  usually won't sort of — in the mission of 
saving trees, we will usually not give
7: And let's see. Oh, and the goal of the projec t should really be for you to do a publishable 
piece of research in machine learning, okay?  
And if you go to the course website, you'll actuall y find a list of the projects that students 
had done last year. And so I'm holding the li st in my hand. Y
